In [1]:
import pandas as pd

In [2]:
tk_data=pd.read_csv('/home/thiri/Documents/RT Projects/Version1/agg/combined_file.csv')

In [3]:
lt_data=pd.read_csv('/home/thiri/Documents/RT Projects/Version1/agg/lt_data.csv')

In [4]:
plan_summary=pd.read_csv('/home/thiri/Documents/RT Projects/Version1/dataset/april_plansummary.csv')

/tmp/ipykernel_40443/37248364.py:1: DtypeWarning: Columns (2,13,18,20,24,26,27,40,46,47,49,50,67,76,83,86,88,91,94,95,96,97,98) have mixed types. Specify dtype option on import or set low_memory=False.
  plan_summary=pd.read_csv('/home/thiri/Documents/RT Projects/Version1/dataset/april_plansummary.csv')


In [5]:
plan_summary.shape

(953917, 102)

### **Loyalty Duration**

In [73]:
tk_data['Date Created'] = pd.to_datetime(tk_data['Date Created'], errors='coerce')

# Merge to bring 'Date Created' into updated_tkOpen_count_data
merged_data = pd.merge(
    lt_data, 
    tk_data[['Customer ID', 'Date Created']], 
    on=['Customer ID'], 
    how='left'
)

# Get the latest (max) date per 'Customer ID', 'Local Service ID', 'Cpe ID'
latest_date_data = (
    merged_data.groupby(['Customer ID'], as_index=False)
    .agg({'Date Created': 'max'})  # Ensures the latest date is selected
)

# Merge back to assign 'Last Ticket Created Date'
last_tk_created_date = pd.merge(
    lt_data, 
    latest_date_data, 
    on=['Customer ID'], 
    how='left'
)

# Rename column for clarity
last_tk_created_date.rename(columns={'Date Created': 'Last Ticket Created Date'}, inplace=True)

last_tk_created_date

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                0    2074729.0                        0   
1                1    2514023.0                        0   
2                2    2288502.0                        0   
3                3    2114773.0                        0   
4                4    2380314.0                        0   
...            ...          ...                      ...   
178320      178320    2553933.0                        1   
178321      178321    2017863.0                        1   
178322      178322    2456162.0                        1   
178323      178323    2377702.0                        1   
178324      178324    2206790.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             1                              0           Low   
1                             1                              0           Low   
2                             2                              0           Low   
3                             0                              0        Medium   
4                             0                              0           Low   
...                         ...                            ...           ...   
178320                        1                              0           Low   
178321                        1                              0           Low   
178322                        1                              0           Low   
178323                        1                              0           Low   
178324                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  
0        0.052234           1.043654       1.937002               2025-01-30  
1        2.070324           1.606961       2.702963               2025-02-24  
2        1.950382           2.320818       3.020289               2025-01-29  
3        3.317627           2.772269       3.317627               2024-10-31  
4        0.962292           1.591817       2.221343               2024-10-14  
...           ...                ...            ...                      ...  
178320   1.032442           1.032442       1.032442               2025-03-01  
178321   0.000590           0.000590       0.000590               2025-03-01  
178322   5.242523           5.242523       5.242523               2025-03-01  
178323   1.104734           1.104734       1.104734               2025-03-01  
178324   1.402350           1.402350       1.402350               2025-03-01  

[178325 rows x 10 columns]

### Check if 'Last Ticket Created Date' is assigned correctly or not

In [74]:
tk_data[tk_data['Customer ID']==2206790][['Customer ID','Date Created']]

Customer ID Date Created
421974    2206790.0   2025-03-01

### Assign Plan Start Date

In [75]:
summary=plan_summary[['Customer ID','Last Used Cpe','Plan Start Date']]

In [76]:
if 'Plan Start Date' in last_tk_created_date.columns:
    last_tk_created_date = last_tk_created_date.drop(columns=['Plan Start Date'])

# Keep only the earliest 'Plan Start Date' for each unique 'Customer ID'
latest_plan_start = summary[['Customer ID','Plan Start Date']].dropna()
latest_plan_start = latest_plan_start.sort_values(by=['Customer ID', 'Plan Start Date']).drop_duplicates(subset=['Customer ID'], keep='first')

# Merge
plan_start_date_data = last_tk_created_date.merge(
    latest_plan_start,
    on=['Customer ID'],
    how='left'
)

plan_start_date_data

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                0    2074729.0                        0   
1                1    2514023.0                        0   
2                2    2288502.0                        0   
3                3    2114773.0                        0   
4                4    2380314.0                        0   
...            ...          ...                      ...   
178320      178320    2553933.0                        1   
178321      178321    2017863.0                        1   
178322      178322    2456162.0                        1   
178323      178323    2377702.0                        1   
178324      178324    2206790.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             1                              0           Low   
1                             1                              0           Low   
2                             2                              0           Low   
3                             0                              0        Medium   
4                             0                              0           Low   
...                         ...                            ...           ...   
178320                        1                              0           Low   
178321                        1                              0           Low   
178322                        1                              0           Low   
178323                        1                              0           Low   
178324                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        0.052234           1.043654       1.937002               2025-01-30   
1        2.070324           1.606961       2.702963               2025-02-24   
2        1.950382           2.320818       3.020289               2025-01-29   
3        3.317627           2.772269       3.317627               2024-10-31   
4        0.962292           1.591817       2.221343               2024-10-14   
...           ...                ...            ...                      ...   
178320   1.032442           1.032442       1.032442               2025-03-01   
178321   0.000590           0.000590       0.000590               2025-03-01   
178322   5.242523           5.242523       5.242523               2025-03-01   
178323   1.104734           1.104734       1.104734               2025-03-01   
178324   1.402350           1.402350       1.402350               2025-03-01   

       Plan Start Date  
0           2018-10-29  
1           2024-02-12  
2           2021-06-22  
3           2019-08-19  
4           2022-02-23  
...                ...  
178320      2024-10-20  
178321      2017-07-22  
178322      2023-01-30  
178323      2022-02-16  
178324      2020-09-10  

[178325 rows x 11 columns]

In [77]:
plan_start_date_data['Plan Start Date'].isna().value_counts()

Plan Start Date
False    178324
True          1
Name: count, dtype: int64

In [78]:
plan_start_date_data[plan_start_date_data['Plan Start Date'].isna()]

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
120442      120442    2551730.0                        0   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
120442                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
120442   0.986667           0.986667       0.986667               2025-01-26   

       Plan Start Date  
120442             NaN

### Calculate Loyalty Duration

In [79]:
plan_start_date_data['Plan Start Date'] = pd.to_datetime(plan_start_date_data['Plan Start Date'], errors='coerce')
plan_start_date_data['Last Ticket Created Date'] = pd.to_datetime(plan_start_date_data['Last Ticket Created Date'], errors='coerce')

plan_start_date_data['Loyalty Duration'] = (
    ((plan_start_date_data['Last Ticket Created Date'] - plan_start_date_data['Plan Start Date']).dt.days / 30.44)*30
).round(2) 

plan_start_date_data

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                0    2074729.0                        0   
1                1    2514023.0                        0   
2                2    2288502.0                        0   
3                3    2114773.0                        0   
4                4    2380314.0                        0   
...            ...          ...                      ...   
178320      178320    2553933.0                        1   
178321      178321    2017863.0                        1   
178322      178322    2456162.0                        1   
178323      178323    2377702.0                        1   
178324      178324    2206790.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             1                              0           Low   
1                             1                              0           Low   
2                             2                              0           Low   
3                             0                              0        Medium   
4                             0                              0           Low   
...                         ...                            ...           ...   
178320                        1                              0           Low   
178321                        1                              0           Low   
178322                        1                              0           Low   
178323                        1                              0           Low   
178324                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        0.052234           1.043654       1.937002               2025-01-30   
1        2.070324           1.606961       2.702963               2025-02-24   
2        1.950382           2.320818       3.020289               2025-01-29   
3        3.317627           2.772269       3.317627               2024-10-31   
4        0.962292           1.591817       2.221343               2024-10-14   
...           ...                ...            ...                      ...   
178320   1.032442           1.032442       1.032442               2025-03-01   
178321   0.000590           0.000590       0.000590               2025-03-01   
178322   5.242523           5.242523       5.242523               2025-03-01   
178323   1.104734           1.104734       1.104734               2025-03-01   
178324   1.402350           1.402350       1.402350               2025-03-01   

       Plan Start Date  Loyalty Duration  
0           2018-10-29           2251.97  
1           2024-02-12            372.54  
2           2021-06-22           1297.96  
3           2019-08-19           1872.54  
4           2022-02-23            950.07  
...                ...               ...  
178320      2024-10-20            130.09  
178321      2017-07-22           2738.83  
178322      2023-01-30            750.00  
178323      2022-02-16           1092.97  
178324      2020-09-10           1609.40  

[178325 rows x 12 columns]

In [80]:
plan_start_date_data

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                0    2074729.0                        0   
1                1    2514023.0                        0   
2                2    2288502.0                        0   
3                3    2114773.0                        0   
4                4    2380314.0                        0   
...            ...          ...                      ...   
178320      178320    2553933.0                        1   
178321      178321    2017863.0                        1   
178322      178322    2456162.0                        1   
178323      178323    2377702.0                        1   
178324      178324    2206790.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             1                              0           Low   
1                             1                              0           Low   
2                             2                              0           Low   
3                             0                              0        Medium   
4                             0                              0           Low   
...                         ...                            ...           ...   
178320                        1                              0           Low   
178321                        1                              0           Low   
178322                        1                              0           Low   
178323                        1                              0           Low   
178324                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        0.052234           1.043654       1.937002               2025-01-30   
1        2.070324           1.606961       2.702963               2025-02-24   
2        1.950382           2.320818       3.020289               2025-01-29   
3        3.317627           2.772269       3.317627               2024-10-31   
4        0.962292           1.591817       2.221343               2024-10-14   
...           ...                ...            ...                      ...   
178320   1.032442           1.032442       1.032442               2025-03-01   
178321   0.000590           0.000590       0.000590               2025-03-01   
178322   5.242523           5.242523       5.242523               2025-03-01   
178323   1.104734           1.104734       1.104734               2025-03-01   
178324   1.402350           1.402350       1.402350               2025-03-01   

       Plan Start Date  Loyalty Duration  
0           2018-10-29           2251.97  
1           2024-02-12            372.54  
2           2021-06-22           1297.96  
3           2019-08-19           1872.54  
4           2022-02-23            950.07  
...                ...               ...  
178320      2024-10-20            130.09  
178321      2017-07-22           2738.83  
178322      2023-01-30            750.00  
178323      2022-02-16           1092.97  
178324      2020-09-10           1609.40  

[178325 rows x 12 columns]

## **On Going Ticket**

In [81]:
tk_data['Status'].value_counts()

Status
closed         398952
in_progress     19060
pending          1368
resolved          845
new               756
re-open           627
re-open-2         148
resolved-2        126
re-open-3          51
resolved-3         16
re-open-4          11
resolved-4         11
resolved-5          5
Name: count, dtype: int64

In [82]:
status_closed=tk_data[tk_data['Status']=='closed']
status_closed.shape

(398952, 15)

In [83]:
status_others=tk_data[tk_data['Status'].isin(['in_progress','new','resolved','pending','re-open','resolved-2','re-open-2','re-open-3','re-open-4','resolved-3','resolved-4','resolved-5'])]
status_others.shape

(23024, 15)

In [84]:
on_going_tks = plan_start_date_data.merge(
    status_others[['Customer ID']].drop_duplicates(),
    left_on=['Customer ID'], 
    right_on=['Customer ID'], 
    how='left',
    indicator=True
)

on_going_tks['On Going Ticket'] = (on_going_tks['_merge'] == 'both').astype(int)
on_going_tks.drop(columns=['_merge'], inplace=True)


In [85]:
on_going_tks

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                0    2074729.0                        0   
1                1    2514023.0                        0   
2                2    2288502.0                        0   
3                3    2114773.0                        0   
4                4    2380314.0                        0   
...            ...          ...                      ...   
178320      178320    2553933.0                        1   
178321      178321    2017863.0                        1   
178322      178322    2456162.0                        1   
178323      178323    2377702.0                        1   
178324      178324    2206790.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             1                              0           Low   
1                             1                              0           Low   
2                             2                              0           Low   
3                             0                              0        Medium   
4                             0                              0           Low   
...                         ...                            ...           ...   
178320                        1                              0           Low   
178321                        1                              0           Low   
178322                        1                              0           Low   
178323                        1                              0           Low   
178324                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        0.052234           1.043654       1.937002               2025-01-30   
1        2.070324           1.606961       2.702963               2025-02-24   
2        1.950382           2.320818       3.020289               2025-01-29   
3        3.317627           2.772269       3.317627               2024-10-31   
4        0.962292           1.591817       2.221343               2024-10-14   
...           ...                ...            ...                      ...   
178320   1.032442           1.032442       1.032442               2025-03-01   
178321   0.000590           0.000590       0.000590               2025-03-01   
178322   5.242523           5.242523       5.242523               2025-03-01   
178323   1.104734           1.104734       1.104734               2025-03-01   
178324   1.402350           1.402350       1.402350               2025-03-01   

       Plan Start Date  Loyalty Duration  On Going Ticket  
0           2018-10-29           2251.97                0  
1           2024-02-12            372.54                0  
2           2021-06-22           1297.96                0  
3           2019-08-19           1872.54                0  
4           2022-02-23            950.07                0  
...                ...               ...              ...  
178320      2024-10-20            130.09                0  
178321      2017-07-22           2738.83                0  
178322      2023-01-30            750.00                0  
178323      2022-02-16           1092.97                0  
178324      2020-09-10           1609.40                0  

[178325 rows x 13 columns]

In [86]:
on_going_tks['On Going Ticket'].value_counts()

On Going Ticket
0    155464
1     22861
Name: count, dtype: int64

### Match with Plan Summary status

In [87]:
plan_summary_status=pd.read_csv(('temp_files/summary_status.csv'))

In [88]:
merged_data=on_going_tks.merge(plan_summary_status, on=['Customer ID'], how='left')

In [89]:
merged_data.shape

(178325, 15)

In [90]:
merged_data

Unnamed: 0_x  Customer ID  Tk Count Within 30 Days  \
0                  0    2074729.0                        0   
1                  1    2514023.0                        0   
2                  2    2288502.0                        0   
3                  3    2114773.0                        0   
4                  4    2380314.0                        0   
...              ...          ...                      ...   
178320        178320    2553933.0                        1   
178321        178321    2017863.0                        1   
178322        178322    2456162.0                        1   
178323        178323    2377702.0                        1   
178324        178324    2206790.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             1                              0           Low   
1                             1                              0           Low   
2                             2                              0           Low   
3                             0                              0        Medium   
4                             0                              0           Low   
...                         ...                            ...           ...   
178320                        1                              0           Low   
178321                        1                              0           Low   
178322                        1                              0           Low   
178323                        1                              0           Low   
178324                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        0.052234           1.043654       1.937002               2025-01-30   
1        2.070324           1.606961       2.702963               2025-02-24   
2        1.950382           2.320818       3.020289               2025-01-29   
3        3.317627           2.772269       3.317627               2024-10-31   
4        0.962292           1.591817       2.221343               2024-10-14   
...           ...                ...            ...                      ...   
178320   1.032442           1.032442       1.032442               2025-03-01   
178321   0.000590           0.000590       0.000590               2025-03-01   
178322   5.242523           5.242523       5.242523               2025-03-01   
178323   1.104734           1.104734       1.104734               2025-03-01   
178324   1.402350           1.402350       1.402350               2025-03-01   

       Plan Start Date  Loyalty Duration  On Going Ticket  Unnamed: 0_y  \
0           2018-10-29           2251.97                0        9715.0   
1           2024-02-12            372.54                0      170966.0   
2           2021-06-22           1297.96                0       76386.0   
3           2019-08-19           1872.54                0       17289.0   
4           2022-02-23            950.07                0      112090.0   
...                ...               ...              ...           ...   
178320      2024-10-20            130.09                0      195160.0   
178321      2017-07-22           2738.83                0        1573.0   
178322      2023-01-30            750.00                0      143088.0   
178323      2022-02-16           1092.97                0           NaN   
178324      2020-09-10           1609.40                0       43712.0   

        Status  
0       ACTIVE  
1       ACTIVE  
2       ACTIVE  
3       ACTIVE  
4       ACTIVE  
...        ...  
178320  ACTIVE  
178321  ACTIVE  
178322  ACTIVE  
178323     NaN  
178324  ACTIVE  

[178325 rows x 15 columns]

In [91]:
merged_data = merged_data.drop(columns=['Unnamed: 0_x'])

In [92]:
merged_data = merged_data.drop(columns=['Unnamed: 0_y'])

In [93]:
merged_data

Customer ID  Tk Count Within 30 Days  Tk Count Within 60 Days  \
0         2074729.0                        0                        1   
1         2514023.0                        0                        1   
2         2288502.0                        0                        2   
3         2114773.0                        0                        0   
4         2380314.0                        0                        0   
...             ...                      ...                      ...   
178320    2553933.0                        1                        1   
178321    2017863.0                        1                        1   
178322    2456162.0                        1                        1   
178323    2377702.0                        1                        1   
178324    2206790.0                        1                        1   

        Previous Termination Attempts Last Priority  Life Time  \
0                                   0           Low   0.052234   
1                                   0           Low   2.070324   
2                                   0           Low   1.950382   
3                                   0        Medium   3.317627   
4                                   0           Low   0.962292   
...                               ...           ...        ...   
178320                              0           Low   1.032442   
178321                              0           Low   0.000590   
178322                              0           Low   5.242523   
178323                              0           Low   1.104734   
178324                              0           Low   1.402350   

        Average Life Time  Max Life Time Last Ticket Created Date  \
0                1.043654       1.937002               2025-01-30   
1                1.606961       2.702963               2025-02-24   
2                2.320818       3.020289               2025-01-29   
3                2.772269       3.317627               2024-10-31   
4                1.591817       2.221343               2024-10-14   
...                   ...            ...                      ...   
178320           1.032442       1.032442               2025-03-01   
178321           0.000590       0.000590               2025-03-01   
178322           5.242523       5.242523               2025-03-01   
178323           1.104734       1.104734               2025-03-01   
178324           1.402350       1.402350               2025-03-01   

       Plan Start Date  Loyalty Duration  On Going Ticket  Status  
0           2018-10-29           2251.97                0  ACTIVE  
1           2024-02-12            372.54                0  ACTIVE  
2           2021-06-22           1297.96                0  ACTIVE  
3           2019-08-19           1872.54                0  ACTIVE  
4           2022-02-23            950.07                0  ACTIVE  
...                ...               ...              ...     ...  
178320      2024-10-20            130.09                0  ACTIVE  
178321      2017-07-22           2738.83                0  ACTIVE  
178322      2023-01-30            750.00                0  ACTIVE  
178323      2022-02-16           1092.97                0     NaN  
178324      2020-09-10           1609.40                0  ACTIVE  

[178325 rows x 13 columns]

In [94]:
merged_data['Status'].value_counts()

Status
ACTIVE        131307
TERMINATED      1273
Name: count, dtype: int64

In [ ]:
merged_data.to_csv('agg/prepared_data.csv')

2325857
For 2436961

In [99]:
merged_data[merged_data['Customer ID']==2436961]

Customer ID  Tk Count Within 30 Days  Tk Count Within 60 Days  \
100439    2436961.0                        1                        2   

        Previous Termination Attempts Last Priority  Life Time  \
100439                              0           Low   2.097303   

        Average Life Time  Max Life Time Last Ticket Created Date  \
100439           1.788053       2.802199               2025-03-19   

       Plan Start Date  Loyalty Duration  On Going Ticket  Status  
100439      2022-10-29             859.4                0  ACTIVE

In [100]:
plan_summary[plan_summary['Customer ID']==2436961]

Plan Start Date Plan Start Month                Devices   Service ID  \
106149      2022-11-29          2022-11  CJ010363,FO1914003787  2436961-001   
707047      2022-10-29          2022-10  CJ010363,FO1914003787  2436961-001   

        Customer ID Customer Type Service Type  Status      Plan  \
106149    2436961.0     MNet Plus    MNet Plus  ACTIVE  Postpaid   
707047    2436961.0     MNet Plus    MNet Plus  ACTIVE  Postpaid   

       Installation Type  ... Additional Note Billing Group Group ID  \
106149    Fiber Internet  ...             NaN           BNY      NaN   
707047    Fiber Internet  ...             NaN           BNY      NaN   

       Group Start Date Group End Date  Terminate Devices Last Status Date  \
106149              NaN            NaN                NaN              NaN   
707047              NaN            NaN                NaN              NaN   

        Last Used Cpe Employment Status  Fpp Refer ID  
106149            NaN               NaN           NaN  
707047       CJ010363               NaN           NaN  

[2 rows x 102 columns]

In [101]:
tk_data[tk_data['Customer ID']==2436961]

Unnamed: 0  Customer ID     CPE ID  Ticket ID Date Created  \
162563       17422    2436961.0  FFA007599    6926885   2024-12-08   
259474       48226    2436961.0  FFA007599    7190449   2025-01-27   
274542       63437    2436961.0  FFA007599    7123120   2025-01-15   
377568       25943    2436961.0  FFA009989    7473620   2025-03-19   

                    Started               Closed             Resolved  \
162563  2024-12-08T14:49:24  2024-12-09T16:04:22  2024-12-08T14:49:36   
259474  2025-01-27T15:35:36  2025-01-29T11:32:38  2025-01-28T11:51:53   
274542  2025-01-15T22:08:34  2025-01-16T12:11:25  2025-01-15T22:08:35   
377568  2025-03-19T15:31:08  2025-03-22T07:57:40  2025-03-20T18:00:37   

            Root Cause Priority Plan Start Date  Status  Life Time  \
162563  Remote Resolve      Low      2022-10-29  closed   1.314236   
259474             NaN   Medium      2022-10-29  closed   2.097303   
274542  Remote Resolve   Medium      2022-10-29  closed   0.938472   
377568             NaN      Low      2022-10-29  closed   2.802199   

        Fiber Olt Name          Ticket Problem  
162563  OVF-TSNX02ZBNY     Unstable Connection  
259474  OVF-TSNX02ZBNY  No Internet Connection  
274542  OVF-TSNX02ZBNY  No Internet Connection  
377568  OVF-TSNX02ZBNY  No Internet Connection